In [ ]:
import smtplib
import schedule
import time
import requests
import os
import logging
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_exchange_rate():
    """Get current USD to INR exchange rate using a free API"""
    try:
        # Using exchangerate-api.com which is free and reliable
        response = requests.get('https://api.exchangerate-api.com/v4/latest/USD', timeout=10)
        response.raise_for_status()
        data = response.json()
        rate = data['rates']['INR']
        logger.info(f"Successfully fetched exchange rate: 1 USD = {rate} INR")
        return rate
    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching exchange rate: {e}")
        return None
    except KeyError as e:
        logger.error(f"Error parsing exchange rate data: {e}")
        return None

def send_email(rate):
    """Send email with current exchange rate"""
    try:
        # Get email configuration from environment variables
        email_address = os.getenv('EMAIL_ADDRESS')
        email_password = os.getenv('EMAIL_PASSWORD')
        recipient_email = os.getenv('RECIPIENT_EMAIL', email_address)
        
        if not email_address or not email_password:
            logger.error("Email credentials not found in environment variables")
            return False
            
        msg = MIMEMultipart()
        msg['From'] = email_address
        msg['To'] = recipient_email
        msg['Subject'] = f'Daily USD to INR Exchange Rate - {datetime.now().strftime("%Y-%m-%d")}'
        
        # Create HTML email body
        body = f"""
        <html>
        <body>
        <h2>Daily Exchange Rate Update</h2>
        <p>Hello,</p>
        <p>Here's today's exchange rate:</p>
        <div style="background-color: #f0f8ff; padding: 15px; border-radius: 5px; margin: 10px 0;">
            <h3 style="color: #2c5aa0;">1 USD = {rate:.4f} INR</h3>
        </div>
        <p>Date: {datetime.now().strftime("%B %d, %Y at %I:%M %p")}</p>
        <p>This is an automated email from your currency tracker.</p>
        </body>
        </html>
        """
        
        msg.attach(MIMEText(body, 'html'))

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(email_address, email_password)
        server.send_message(msg)
        server.quit()
        
        logger.info(f"Email sent successfully to {recipient_email}")
        return True
        
    except Exception as e:
        logger.error(f"Error sending email: {e}")
        return False

def job():
    """Main job function to get rate and send email"""
    logger.info("Starting daily exchange rate job")
    rate = get_exchange_rate()
    
    if rate is not None:
        success = send_email(rate)
        if success:
            logger.info("Daily exchange rate job completed successfully")
        else:
            logger.error("Failed to send email")
    else:
        logger.error("Failed to fetch exchange rate")

# Schedule the job for weekdays at 8:00 AM
schedule.every().monday.at("08:00").do(job)
schedule.every().tuesday.at("08:00").do(job)
schedule.every().wednesday.at("08:00").do(job)
schedule.every().thursday.at("08:00").do(job)
schedule.every().friday.at("08:00").do(job)

logger.info("Exchange rate scheduler started. Will send emails on weekdays at 8:00 AM")

# Main loop
try:
    while True:
        schedule.run_pending()
        time.sleep(60)  # Check every minute instead of every second
except KeyboardInterrupt:
    logger.info("Scheduler stopped by user")
except Exception as e:
    logger.error(f"Unexpected error: {e}")